In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from __future__ import print_function

import os
import sys
import matplotlib.pyplot as plt

from keras.applications.imagenet_utils import preprocess_input
from keras.models import load_model

sys.path.append('..')
from utils import display_examples, RotNetDataGenerator, angle_error
from data.street_view import get_filenames

## Load file names

In [3]:
street_view_dir = os.path.join('..', 'data', 'street_view')

if os.path.exists(street_view_dir):
    test_filenames = get_filenames(street_view_dir)[1]
else:
    test_examples_path = os.path.join('..', 'data', 'test_examples')
    test_filenames = [
        os.path.join(test_examples_path, f) for f in os.listdir(test_examples_path)
    ]

In [4]:
import keras.backend as K

In [5]:
def custom_loss(y_true, y_pred):
    entropy = K.categorical_crossentropy(y_true, y_pred)
    return entropy + angle_error(y_true, y_pred)

## Load model

In [6]:
model_location = os.path.join('..', 'models', 'rotnet_street_view_resnet50_2.keras')
model = load_model(model_location, custom_objects={'angle_error': angle_error})

## Evaluate model

In [7]:
batch_size = 64
out = model.evaluate_generator(
    RotNetDataGenerator(
        test_filenames,
        input_shape=(224, 224, 3),
        batch_size=batch_size,
        preprocess_func=preprocess_input,
        crop_center=True,
        crop_largest_rect=True,
        shuffle=True
    ),
    steps=len(test_filenames) / batch_size
)

print('Test loss:', out[0])
print('Test angle error:', out[1])

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_4224\420072282.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  out = model.evaluate_generator(


Could not read image ..\data\street_view\929.png
Could not read image ..\data\street_view\9695.png
Could not read image ..\data\street_view\9128.png
Could not read image ..\data\street_view\924.png
Could not read image ..\data\street_view\9110.png
Could not read image ..\data\street_view\9985.png
Could not read image ..\data\street_view\9465.png
Could not read image ..\data\street_view\9667.png
Could not read image ..\data\street_view\9310.png
Could not read image ..\data\street_view\9717.png
Could not read image ..\data\street_view\9914.png
Could not read image ..\data\street_view\9502.png
Could not read image ..\data\street_view\9731.png
Could not read image ..\data\street_view\9656.png
Could not read image ..\data\street_view\9894.png
Could not read image ..\data\street_view\9202.png
Could not read image ..\data\street_view\9517.png
Could not read image ..\data\street_view\9256.png
Could not read image ..\data\street_view\9179.png
Could not read image ..\data\street_view\9837.png
Co

UnimplementedError: Graph execution error:

Detected at node 'model/conv1_conv/Conv2D' defined at (most recent call last):
    File "c:\Users\ADMIN\anaconda3\envs\ImageOrientationCorrection\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\ADMIN\anaconda3\envs\ImageOrientationCorrection\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\ADMIN\anaconda3\envs\ImageOrientationCorrection\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\ADMIN\anaconda3\envs\ImageOrientationCorrection\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\ADMIN\anaconda3\envs\ImageOrientationCorrection\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\ADMIN\anaconda3\envs\ImageOrientationCorrection\lib\site-packages\tornado\p

## Display examples

In [ ]:
num_images = 10

display_examples(
    model, 
    test_filenames,
    num_images=num_images,
    size=(224, 224),
    crop_center=True,
    crop_largest_rect=True,
    preprocess_func=preprocess_input,
)

NameError: name 'display_examples' is not defined